In [1]:
import numpy as np
import pandas as pd
import plotly_express as px
import plotly.graph_objects as go
import scipy.signal as signal
import dfUtils
from const import left_insole_truncate_mapping
from const import right_insole_truncate_mapping

# Read insole data and forceplate data

* insole data format - csv

* forceplate data format - xlsx

In [ ]:
insoleDataFile = r"D:\code\Exoskeleton\data\20241014_calibration\original\insoleR10ms.csv"
forceplateDataFile = r"D:\code\Exoskeleton\data\20241013_calibration\original\forceplateR10ms.xlsx" #  your file path

insoleDatadf = pd.read_csv(insoleDataFile, sep=',')
forceplateDatadf = pd.read_excel(forceplateDataFile)

# Process the original data

* 

## Process the original insole data

* 

In [ ]:
samplingRate = 50
cutoffFreq = 5
nyquist = 0.5 * samplingRate
insoleUnit = 18
normalCutoff = cutoffFreq / nyquist
b, a = signal.butter(4, normalCutoff, btype='low', analog=False)

for i in range(insoleUnit):
    insoleDatadf[f'I{i}_lowpass'] = signal.filtfilt(b, a, insoleDatadf[f'I{i}']) # lowpass filter
    insoleDatadf[f'I{i}_zeroed'] = np.where(insoleDatadf[f'I{i}_lowpass'] < left_insole_truncate_mapping[f'I{i}'], 0, insoleDatadf[f'I{i}_lowpass'])

columnsToSum = [f'I{i}_zeroed' for i in range(insoleUnit)]
insoleDatadf['Sum'] = insoleDatadf[columnsToSum].sum(axis=1)

insoleDatadf = insoleDatadf.reset_index()

## Process the original forceplate data

* 

In [ ]:
samplingRate = 1200
cutoffFreq = 20
nyquist = 0.5 * samplingRate
normalCutoff = cutoffFreq / nyquist
b, a = signal.butter(4, normalCutoff, btype='low', analog=False)
forceplateDatadf['FZ1_lowpass'] = signal.filtfilt(b, a, forceplateDatadf['FZ1']) ## 左腿
# forceplateDatadf['FZ2_lowpass'] = signal.filtfilt(b, a, forceplateDatadf['FZ2']) ## 右腿

threshold = 20 # 小于threshold都认为没有力
forceplateDatadf['FZ1_zeroed'] = np.where(forceplateDatadf['FZ1_lowpass'] < threshold, 0, forceplateDatadf['FZ1_lowpass'])
# forceplateDatadf['FZ2_zeroed'] = np.where(forceplateDatadf['FZ2_lowpass'] < threshold, 0, forceplateDatadf['FZ2_lowpass'])

forceplateDatadf = forceplateDatadf.reset_index()

# Visualization

In [ ]:
fig = go.Figure()

# visualize insole data
for i in range(insoleUnit):
    dfUtils.visualizeData(fig, insoleDatadf['index'].values, insoleDatadf[f'I{i}'].values, lineName=f'I{i}')
    dfUtils.visualizeData(fig, insoleDatadf['index'].values, insoleDatadf[f'I{i}_lowpass'].values, lineName=f'I{i}_lowpass')
    dfUtils.visualizeData(fig, insoleDatadf['index'].values, insoleDatadf[f'I{i}_zeroed'].values, lineName=f'I{i}_zeroed')

# visualize forceplate data
dfUtils.visualizeData(fig, forceplateDatadf['index'].values, forceplateDatadf['FZ1'].values, lineName='Force plate force before lowpass')
dfUtils.visualizeData(fig, forceplateDatadf['index'].values, forceplateDatadf['FZ1_lowpass'].values, lineName='Force plate force after lowpass')
dfUtils.visualizeData(fig, forceplateDatadf['index'].values, forceplateDatadf['FZ1_zeroed'].values, lineName='Force plate force after zeroed')

# dfUtils.visualizeData(fig, forceplateDatadf['index'].values, forceplateDatadf['FZ2'].values, lineName='Force plate force before lowpass')
# dfUtils.visualizeData(fig, forceplateDatadf['index'].values, forceplateDatadf['FZ2_lowpass'].values, lineName='Force plate force after lowpass')
# dfUtils.visualizeData(fig, forceplateDatadf['index'].values, forceplateDatadf['FZ2_zeroed'].values, lineName='Force plate force after zeroed')

fig.show()

# Save file

In [ ]:
insoleSaveFile = r"D:\code\Exoskeleton\data\20241014_calibration\clipped\insoleR10ms_clipped.xlsx"
forceplateSaveFile = r"D:\code\Exoskeleton\data\20241014_calibration\clipped\forceplateR10ms_clipped.xlsx"

# extract especial row
startRowIndex = 908
endRowIndex  = 8945

dfUtils.clipDataAndSave(insoleDatadf, startRowIndex, endRowIndex, insoleSaveFile)
# dfUtils.clipDataAndSave(forceplateDatadf, startRowIndex, endRowIndex, forceplateSaveFile)